In [1]:
import nltk
from nltk import PCFG
import numpy

In [2]:

#sentence = ["movies", "directed", "by", "mel", "brooks"]
#sentence = nltk.corpus.treebank.tagged_sents()[22]

#sentence = "adventure movies between 2000 and 2015 featuring performances by daniel craig"
sentence = "movies directed by craig"
# #sentence = "all movies where Vin Diesel acted from the 1990s"
#sentence = "movies directed by mel brooks\r\n"
#sentence = "action movies with Jackie Chan\r\n"
#print sentence

In [3]:
    sentence = nltk.word_tokenize(sentence)
    
    #sentence = [nltk.word_tokenize(sent) for sent in sentence]
    #sentence = nltk.word_tokenize(sentence)
    #sentence = [nltk.pos_tag(sent) for sent in sentence]
    #sentence = nltk.pos_tag(sentence)
    print sentence

['movies', 'directed', 'by', 'craig']


In [4]:
#TV = transitive verb
#IV = intransitive verb
# DatV = Dative verb
grammar = PCFG.fromstring("""
          S    -> NP VP                   [1.0]
          VP   -> TV IN NP                [0.8]
          VP   -> IV                      [0.1]
          VP   -> DatV NP NP              [0.1]  
          IN   -> 'by'                    [1.0]
          TV   -> 'directed'              [1.0]
          IV   -> 'directed'              [1.0]
          DatV -> 'gave'                  [1.0]
          NP   -> 'movies'      [0.5]
          NP   -> 'craig'          [0.5]
            """)


In [5]:
viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sentence):
        print tree

(S (NP movies) (VP (TV directed) (IN by) (NP craig))) (p=0.2)


In [7]:
# 

from nltk import PCFG
toy_pcfg1 = PCFG.fromstring("""
    S -> NP VP [1.0]
    NP -> Det N [0.5] | NP PP [0.25] | 'John' [0.1] | 'I' [0.15]
    Det -> 'the' [0.8] | 'my' [0.2]
    N -> 'man' [0.5] | 'telescope' [0.5]
    VP -> VP PP [0.1] | V NP [0.7] | V [0.2]
    V -> 'ate' [0.35] | 'saw' [0.65]
    PP -> P NP [1.0]
    P -> 'with' [0.61] | 'under' [0.39]
""")

In [8]:
from nltk.corpus import treebank
from nltk import treetransforms
from nltk import induce_pcfg
from nltk.parse import pchart

pcfg_prods = toy_pcfg1.productions()

pcfg_prod = pcfg_prods[2]
print('A PCFG production:', pcfg_prod)
print('pcfg_prod.lhs()  =>', pcfg_prod.lhs())
print('pcfg_prod.rhs()  =>', pcfg_prod.rhs())
print('pcfg_prod.prob() =>', pcfg_prod.prob())

('A PCFG production:', NP -> NP PP [0.25])
('pcfg_prod.lhs()  =>', NP)
('pcfg_prod.rhs()  =>', (NP, PP))
('pcfg_prod.prob() =>', 0.25)


In [9]:
# extract productions from three trees and induce the PCFG
print("Induce PCFG grammar from treebank data:")

productions = []
for item in treebank.fileids()[:2]:
  for tree in treebank.parsed_sents(item):
    # perform optional tree transformations, e.g.:
    tree.collapse_unary(collapsePOS = False)# Remove branches A-B-C into A-B+C
    tree.chomsky_normal_form(horzMarkov = 2)# Remove A->(B,C,D) into A->B,C+D->D
    productions += tree.productions()

Induce PCFG grammar from treebank data:


In [10]:
from nltk import Nonterminal
S = Nonterminal('S')
grammar = induce_pcfg(S, productions)
print(grammar)

Grammar with 86 productions (start state = S)
    NNP -> 'Agnew' [0.0714286]
    JJ -> 'industrial' [0.142857]
    VP -> VBZ NP-PRD [0.2]
    VP|<PP-CLR-NP-TMP> -> PP-CLR NP-TMP [1.0]
    NP -> NN [0.0666667]
    NP -> NNP NNP [0.2]
    VP -> MD VP [0.2]
    NP-TMP -> NNP CD [1.0]
    NP-SBJ -> -NONE- [0.333333]
    . -> '.' [1.0]
    NN -> 'group' [0.142857]
    NN -> 'conglomerate' [0.142857]
    DT -> 'a' [0.4]
    IN -> 'of' [0.75]
    NP|<NNP-NNP> -> NNP NNP [0.5]
    CC -> 'and' [1.0]
    NP|<VBG-NN> -> VBG NN [1.0]
    NNP -> 'Rudolph' [0.0714286]
    NP -> NP PP [0.0666667]
    NNP -> 'PLC' [0.0714286]
    NP-SBJ|<,-ADJP> -> , NP-SBJ|<ADJP-,> [1.0]
    CD -> '61' [0.333333]
    PP -> IN NP [1.0]
    NP|<JJ-NN> -> JJ NN [1.0]
    JJ -> 'nonexecutive' [0.285714]
    NNP -> 'N.V.' [0.0714286]
    NNS -> 'years' [1.0]
    VP -> VB VP|<NP-PP-CLR> [0.2]
    NNP -> 'Dutch' [0.0714286]
    NNP -> 'Mr.' [0.0714286]
    NP -> DT NP|<JJ-JJ> [0.0666667]
    NP -> DT NP|<NNP-VBG> [0.0666667

In [11]:
print("Parse sentence using induced grammar:")

parser = pchart.InsideChartParser(grammar)
parser.trace(3)

sent = treebank.parsed_sents('wsj_0001.mrg')[0].leaves()
print(sent)

for parse in parser.parse(sent):
  print(parse)

4286]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NP|<NNP-VBG> -> * NNP NP|<VBG-NN> [1.0]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NP-TMP -> * NNP CD [1.0]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NP|<NNP-NNP> -> * NNP NP|<NNP-NNP> [0.5]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NP|<NNP-NNP> -> * NNP NNP [0.5]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NP-SBJ -> * NNP NNP [0.333333333333]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NP -> * NNP NNP [0.2]
  |. . . . . . . . . . . . . . . [-> . .| [15:16] NP|<NNP-VBG> -> NNP * NP|<VBG-NN> [0.0714285714286]
  |. . . . . . . . . . . . . . . [-> . .| [15:16] NP-TMP -> NNP * CD [0.0714285714286]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NNP -> * 'Nov.' [0.0714285714286]
  |. . . . . . . . . . . . . . . > . . .| [15:15] NP -> * NNP NP|<NNP-NNP> [0.0666666666667]
  |> . . . . . . . . . . . . . . . . . .| [0:0] NP -> * NNP NP|<NNP-NNP> [0.0666666666667]
  |. . . . . . . . . . > . . . . . . . .